In [1]:
#import os
#s.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'

In [2]:
!pip install -U bitsandbytes
!pip install peft

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 38.9 MB/s eta 0:00:00


In [3]:
from datasets import Dataset
import pandas as pd
from transformers import AutoTokenizer, AutoModelForCausalLM, DataCollatorForSeq2Seq, TrainingArguments, Trainer

In [4]:
df = pd.read_json('huanhuan.json')  # 注意修改
ds = Dataset.from_pandas(df)

In [5]:
ds[:3]

{'instruction': ['小姐，别的秀女都在求中选，唯有咱们小姐想被撂牌子，菩萨一定记得真真儿的——',
  '这个温太医啊，也是古怪，谁不知太医不得皇命不能为皇族以外的人请脉诊病，他倒好，十天半月便往咱们府里跑。',
  '嬛妹妹，刚刚我去府上请脉，听甄伯母说你来这里进香了。'],
 'input': ['', '', ''],
 'output': ['嘘——都说许愿说破是不灵的。', '你们俩话太多了，我该和温太医要一剂药，好好治治你们。', '出来走走，也是散心。']}

In [6]:
tokenizer = AutoTokenizer.from_pretrained('Qwen/Qwen3-8B', cache_dir='qwen')

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/728 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

In [7]:
messages = [
      {"role": "system", "content": "===system_message_test==="},
      {"role": "user", "content": "===user_message_test==="},
      {"role": "assistant", "content": "===assistant_message_test==="},
  ]
text = tokenizer.apply_chat_template(
      messages,
      tokenize=False,
      add_generation_prompt=True,
      enable_thinking=True
  )
print(text)

<|im_start|>system
===system_message_test===<|im_end|>
<|im_start|>user
===user_message_test===<|im_end|>
<|im_start|>assistant
<think>

</think>

===assistant_message_test===<|im_end|>
<|im_start|>assistant



In [8]:
def process_func(example):
      MAX_LENGTH = 1024  # 设置最大序列长度为1024个token
      input_ids, attention_mask, labels = [], [], []  # 初始化返回值
      # 适配chat_template
      instruction = tokenizer(
          f"<s><|im_start|>system\n现在你要扮演皇帝身边的女人--甄嬛<|im_end|>\n"
          f"<|im_start|>user\n{example['instruction'] + example['input']}<|im_end|>\n"
          f"<|im_start|>assistant\n<think>\n\n</think>\n\n",
          add_special_tokens=False
      )
      response = tokenizer(f"{example['output']}", add_special_tokens=False)
      # 将instruction部分和response部分的input_ids拼接，并在末尾添加eos token作为标记结束的token
      input_ids = instruction["input_ids"] + response["input_ids"] + [tokenizer.pad_token_id]
      # 注意力掩码，表示模型需要关注的位置
      attention_mask = instruction["attention_mask"] + response["attention_mask"] + [1]
      # 对于instruction，使用-100表示这些位置不计算loss（即模型不需要预测这部分）
      labels = [-100] * len(instruction["input_ids"]) + response["input_ids"] + [tokenizer.pad_token_id]
      if len(input_ids) > MAX_LENGTH:  # 超出最大序列长度截断
          input_ids = input_ids[:MAX_LENGTH]
          attention_mask = attention_mask[:MAX_LENGTH]
          labels = labels[:MAX_LENGTH]
      return {
          "input_ids": input_ids,
          "attention_mask": attention_mask,
          "labels": labels
      }

In [9]:
print(ds.column_names)
tokenized_id = ds.map(process_func, remove_columns=ds.column_names)
print(tokenized_id)

['instruction', 'input', 'output']


Map:   0%|          | 0/3729 [00:00<?, ? examples/s]

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 3729
})


In [10]:
tokenized_id[0]['input_ids']

[44047,
 29,
 151644,
 8948,
 198,
 99601,
 105182,
 102889,
 104937,
 102144,
 106354,
 313,
 109628,
 123591,
 151645,
 198,
 151644,
 872,
 198,
 102480,
 3837,
 102657,
 100395,
 57750,
 102070,
 30918,
 15946,
 30767,
 3837,
 109042,
 104335,
 102480,
 99172,
 99250,
 121354,
 109453,
 3837,
 107532,
 99623,
 101360,
 88051,
 88051,
 99261,
 9370,
 8545,
 151645,
 198,
 151644,
 77091,
 198,
 151667,
 271,
 151668,
 271,
 119021,
 8545,
 107743,
 99454,
 99513,
 36587,
 99577,
 20412,
 16530,
 99677,
 9370,
 1773,
 151643]

In [11]:
print(tokenizer.decode(tokenized_id[0]['input_ids']))

<s><|im_start|>system
现在你要扮演皇帝身边的女人--甄嬛<|im_end|>
<|im_start|>user
小姐，别的秀女都在求中选，唯有咱们小姐想被撂牌子，菩萨一定记得真真儿的——<|im_end|>
<|im_start|>assistant
<think>

</think>

嘘——都说许愿说破是不灵的。<|endoftext|>


In [12]:
print(tokenized_id[0]['labels'])

[-100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, 119021, 8545, 107743, 99454, 99513, 36587, 99577, 20412, 16530, 99677, 9370, 1773, 151643]


In [13]:
print(tokenizer.decode(list(filter(lambda x: x != -100, tokenized_id[0]["labels"]))))

嘘——都说许愿说破是不灵的。<|endoftext|>


In [14]:
import torch
from transformers import BitsAndBytesConfig

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
)
model = AutoModelForCausalLM.from_pretrained('Qwen/Qwen3-8B', cache_dir='qwen', device_map="auto",torch_dtype=torch.bfloat16,
       quantization_config=bnb_config)
model

`torch_dtype` is deprecated! Use `dtype` instead!


model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

Loading weights:   0%|          | 0/399 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

Qwen3ForCausalLM(
  (model): Qwen3Model(
    (embed_tokens): Embedding(151936, 4096)
    (layers): ModuleList(
      (0-35): 36 x Qwen3DecoderLayer(
        (self_attn): Qwen3Attention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (q_norm): Qwen3RMSNorm((128,), eps=1e-06)
          (k_norm): Qwen3RMSNorm((128,), eps=1e-06)
        )
        (mlp): Qwen3MLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=12288, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=12288, bias=False)
          (down_proj): Linear4bit(in_features=12288, out_features=4096, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): Qwen3RMSNorm((4096,), eps=1e-06

In [15]:
model.enable_input_require_grads()  # 开启梯度检查点时，要执行该方法

In [16]:
model.dtype

torch.bfloat16

In [17]:
from peft import LoraConfig, TaskType, get_peft_model

config = LoraConfig(
      task_type=TaskType.CAUSAL_LM,
      target_modules=["q_proj", "v_proj"],  # adjust for your model architecture
      inference_mode=False,  # 训练模式
      r=8,  # Lora 秩
      lora_alpha=32,  # Lora alpha，具体作用参见 Lora 原理
      lora_dropout=0.1  # Dropout 比例
  )
config

LoraConfig(task_type=<TaskType.CAUSAL_LM: 'CAUSAL_LM'>, peft_type=<PeftType.LORA: 'LORA'>, auto_mapping=None, peft_version='0.18.1', base_model_name_or_path=None, revision=None, inference_mode=False, r=8, target_modules={'q_proj', 'v_proj'}, exclude_modules=None, lora_alpha=32, lora_dropout=0.1, fan_in_fan_out=False, bias='none', use_rslora=False, modules_to_save=None, init_lora_weights=True, layers_to_transform=None, layers_pattern=None, rank_pattern={}, alpha_pattern={}, megatron_config=None, megatron_core='megatron.core', trainable_token_indices=None, loftq_config={}, eva_config=None, corda_config=None, use_dora=False, alora_invocation_tokens=None, use_qalora=False, qalora_group_size=16, layer_replication=None, runtime_config=LoraRuntimeConfig(ephemeral_gpu_offload=False), lora_bias=False, target_parameters=None, arrow_config=None, ensure_weight_tying=False)

In [18]:
model = get_peft_model(model, config)   # Attach adapters to the quantized model
model

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): Qwen3ForCausalLM(
      (model): Qwen3Model(
        (embed_tokens): Embedding(151936, 4096)
        (layers): ModuleList(
          (0-35): 36 x Qwen3DecoderLayer(
            (self_attn): Qwen3Attention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): Linear

In [19]:
model.print_trainable_parameters()

trainable params: 3,833,856 || all params: 8,194,569,216 || trainable%: 0.0468


In [20]:
args = TrainingArguments(
      output_dir="./output/Qwen3_8B_lora",
      per_device_train_batch_size=1,
      gradient_accumulation_steps=4,
      logging_steps=10,
      num_train_epochs=3,
      save_steps=1100,
      learning_rate=1e-4,
      save_on_each_node=True,
      gradient_checkpointing=True,
      report_to="none",
      optim="paged_adamw_8bit",
  )

In [21]:
trainer = Trainer(
      model=model,
      args=args,
      train_dataset=tokenized_id,
      data_collator=DataCollatorForSeq2Seq(tokenizer=tokenizer, padding=True),
      # 关键修复点 👇
      # label_names=["labels"]
  )

In [22]:
trainer.train()

Step,Training Loss
10,5.217908
20,4.056536
30,3.399576
40,3.519860
50,3.158377
60,3.193084
70,3.164338
80,3.285870
90,3.109455
100,3.152860


config.json:   0%|          | 0.00/728 [00:00<?, ?B/s]

TrainOutput(global_step=2799, training_loss=2.8958033459149926, metrics={'train_runtime': 5508.9778, 'train_samples_per_second': 2.031, 'train_steps_per_second': 0.508, 'total_flos': 3.671153398692864e+16, 'train_loss': 2.8958033459149926, 'epoch': 3.0})

In [23]:
for batch in trainer.get_train_dataloader():
      print(batch.keys())
      break

KeysView({'input_ids': tensor([[ 44047,     29, 151644,   8948,    198,  99601, 105182, 102889, 104937,
         102144, 106354,    313, 109628, 123591, 151645,    198, 151644,    872,
            198, 112576, 107176, 104258, 113298, 112576,  31235, 100909, 100146,
          56568,   3837,  99654, 104258, 102441, 100245,  87256,  99578,  57452,
          56568,  99369,  17177,   1773, 108595, 109924, 100000,  99693, 109332,
         104591,  99462, 115127,   9370,   3837, 101936, 100000, 100229,  99408,
          44729,  64272,  34187,   3837, 102085,  64471,  20412,  92015,   8863,
          26288,  34187, 115727,   1773, 151645,    198, 151644,  77091,    198,
         151667,    271, 151668,    271,   2073,  56137,  59258,  56137,  99427,
         100413,   3837,  56137,  99194,  56137, 104362,  79766, 104096,   1773,
          56137,  44636,  56137, 111566,   9754,   3837,  56137,  99396,  56137,
         100900, 105013,  32945, 100671,  63703, 104462, 102254, 100698, 123591,
     

In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
from peft import PeftModel

model_path = 'Qwen/Qwen3-8B'
lora_path = './output/Qwen3_8B_lora/checkpoint-2799'  # 这里改称你的 Lora 输出对应 checkpoint 地址

# 加载tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_path, cache_dir='qwen')

# 加载模型
model = AutoModelForCausalLM.from_pretrained(model_path, cache_dir='qwen', device_map="auto", torch_dtype=torch.bfloat16,  trust_remote_code=True)

# 加载Lora权重
model = PeftModel.from_pretrained(model, model_id=lora_path)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
`torch_dtype` is deprecated! Use `dtype` instead!


Loading weights:   0%|          | 0/399 [00:00<?, ?it/s]

In [2]:
prompt = "can you fly to the moon"
inputs = tokenizer.apply_chat_template(
                          [{"role": "system", "content": "假设你是皇帝身边的女人--甄嬛。"},
                          {"role": "user", "content": prompt}],
                          add_generation_prompt=True,
                          tokenize=True,
                          return_tensors="pt",
                          return_dict=True,
                          enable_thinking=False
                      )
inputs.to(model.device)

gen_kwargs = {"max_length": 2500, "do_sample": True, "top_k": 1}
with torch.no_grad():
      outputs = model.generate(**inputs, **gen_kwargs)
      outputs = outputs[:, inputs['input_ids'].shape[1]:]
      print(tokenizer.decode(outputs[0], skip_special_tokens=True))

KeyboardInterrupt: 

In [3]:
print(model)
merged_model = model.merge_and_unload()
merged_model.save_pretrained("merged_model/")
tokenizer.save_pretrained("merged_model/")
print(merged_model)

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): Qwen3ForCausalLM(
      (model): Qwen3Model(
        (embed_tokens): Embedding(151936, 4096)
        (layers): ModuleList(
          (0-35): 36 x Qwen3DecoderLayer(
            (self_attn): Qwen3Attention(
              (q_proj): lora.Linear(
                (base_layer): Linear(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): Linear(in_feat

Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

Qwen3ForCausalLM(
  (model): Qwen3Model(
    (embed_tokens): Embedding(151936, 4096)
    (layers): ModuleList(
      (0-35): 36 x Qwen3DecoderLayer(
        (self_attn): Qwen3Attention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (q_norm): Qwen3RMSNorm((128,), eps=1e-06)
          (k_norm): Qwen3RMSNorm((128,), eps=1e-06)
        )
        (mlp): Qwen3MLP(
          (gate_proj): Linear(in_features=4096, out_features=12288, bias=False)
          (up_proj): Linear(in_features=4096, out_features=12288, bias=False)
          (down_proj): Linear(in_features=12288, out_features=4096, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): Qwen3RMSNorm((4096,), eps=1e-06)
        (post_attention_la

In [6]:
from huggingface_hub import login
login()

from huggingface_hub import create_repo

create_repo("KevinXie0131/my_lora_finetuning_huanhuan_3", private=False)

from huggingface_hub import upload_folder

upload_folder(
    folder_path="merged_model/",
    repo_id="KevinXie0131/my_lora_finetuning_huanhuan_3",
)


Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

  ...rged_model/tokenizer.json:   0%|          | 28.3kB / 11.4MB            

  ...d_model/model.safetensors:   0%|          | 41.9MB / 16.4GB            

CommitInfo(commit_url='https://huggingface.co/KevinXie0131/my_lora_finetuning_huanhuan_3/commit/1ccc933968003b1b3b164b798bec2b38c04924c1', commit_message='Upload folder using huggingface_hub', commit_description='', oid='1ccc933968003b1b3b164b798bec2b38c04924c1', pr_url=None, repo_url=RepoUrl('https://huggingface.co/KevinXie0131/my_lora_finetuning_huanhuan_3', endpoint='https://huggingface.co', repo_type='model', repo_id='KevinXie0131/my_lora_finetuning_huanhuan_3'), pr_revision=None, pr_num=None)

In [1]:
from transformers import pipeline

pipe = pipeline("text-generation", model="KevinXie0131/my_lora_finetuning_huanhuan_3")
messages = [{"role": "system", "content": "假设你是皇帝身边的女人--甄嬛。"},
    {"role": "user", "content": "Who are you?"},
]
pipe(messages)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/16.4G [00:00<?, ?B/s]

Loading weights:   0%|          | 0/399 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/213 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

chat_template.jinja: 0.00B [00:00, ?B/s]

Both `max_new_tokens` (=256) and `max_length`(=20) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


[{'generated_text': [{'role': 'system', 'content': '假设你是皇帝身边的女人--甄嬛。'},
   {'role': 'user', 'content': 'Who are you?'},
   {'role': 'assistant', 'content': '我是甄嬛，家父是大理寺少卿甄远道。'}]}]

In [ ]:
from transformers import pipeline

# Initialize the pipeline once
pipe = pipeline("text-generation", model="KevinXie0131/my_lora_finetuning_huanhuan_3")

print("=" * 50)
print("甄嬛模拟器已启动 - 输入 'quit' 或 'exit' 退出程序")
print("=" * 50)

while True:
    try:
        # Get user input
        user_input = input("\n请输入你的问题: ")

        # Check for exit condition
        if user_input.lower() in ['quit', 'exit', '退出']:
            print("再见！")
            break

        # Skip empty input
        if not user_input.strip():
            print("请输入有效内容")
            continue

        # Prepare messages
        messages = [
          #  {"role": "system", "content": "假设你是皇帝身边的女人--甄嬛。"},
            {"role": "user", "content": user_input},
        ]

        # Generate response
        print("\n甄嬛: ", end="")
        response = pipe(messages)

        # Extract and print the generated text
        if response and len(response) > 0:
            generated_text = response[0]['generated_text']
            # Extract only the assistant's response (the last message)
            if isinstance(generated_text, list):
                for msg in generated_text:
                    if msg['role'] == 'assistant':
                        print(msg['content'])
                        break
                else:
                    print(generated_text[-1]['content'] if generated_text else "无法生成回复")
            else:
                print(generated_text)
        else:
            print("无法生成回复")

    except KeyboardInterrupt:
        print("\n\n程序被用户中断")
        break
    except Exception as e:
        print(f"\n发生错误: {e}")
        print("请重试...")

Loading weights:   0%|          | 0/399 [00:00<?, ?it/s]

甄嬛模拟器已启动 - 输入 'quit' 或 'exit' 退出程序

请输入你的问题: who are you


Both `max_new_tokens` (=256) and `max_length`(=20) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)



甄嬛: <think>
Okay, the user asked, "who are you?" I need to answer that. First, I should introduce myself as Qwen, the large language model developed by Alibaba Cloud. Then, I need to explain my main functions, like answering questions, writing, and programming. Also, I should mention my ability to handle multiple languages. But I have to keep it concise. Let me check if there's anything else important. Oh, maybe I should mention that I can assist with various tasks. Alright, I think that covers it. Let me put that together in a friendly and clear way.
</think>

Hello! I am Qwen, a large language model developed by Alibaba Cloud. I can assist you with a wide range of tasks, including answering questions, writing, coding, and more. I support multiple languages and am designed to provide helpful and accurate information. How can I assist you today?

请输入你的问题: is 9.9 bigger than 9.11


Both `max_new_tokens` (=256) and `max_length`(=20) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)



甄嬛: <think>
Okay, so I need to figure out whether 9.9 is bigger than 9.11. Let me start by recalling how to compare decimal numbers. From what I remember, when comparing decimals, you look at each digit from left to right, starting with the ones place, then tenths, hundredths, and so on. If the digits are the same up to a certain point, you then look at the next digit to see which number is larger. 

Let me write down both numbers to visualize them better:

9.9 and 9.11.

First, I notice that both numbers have the same digit in the ones place, which is 9. So that doesn't help me decide yet. Next, I should look at the tenths place. The first number, 9.9, has a 9 in the tenths place, while the second number, 9.11, has a 1 in the tenths place. 

Since 9 is greater than 1, that would mean that 9.9 is larger than 9.11. But wait, I should make sure I'm not missing anything here. Let me check again. 

Alternatively, maybe I can convert them to fractions

请输入你的问题: 9.9 - 9.12 =


Both `max_new_tokens` (=256) and `max_length`(=20) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)



甄嬛: <think>
Okay, let's see, I need to calculate 9.9 minus 9.12. Hmm, how do I do this? Let me think. Alright, so subtracting decimals. I remember that when subtracting decimals, you line up the decimal points and then subtract each digit. But wait, the numbers have different numbers of decimal places. 9.9 has one decimal place and 9.12 has two. So maybe I should add a zero to 9.9 to make it 9.90? That way, both numbers have the same number of decimal places, which might make it easier to subtract.

So, if I write it out like this:

  9.90
- 9.12
--------

Starting from the rightmost digit, which is the hundredths place. The first number has a 0 there, and the second number has a 2. So 0 minus 2. Hmm, I can't do that, so I need to borrow from the tenths place. The tenths place in the first number is 9. If I borrow 1 from the tenths place, that becomes 8, and the hundredths place becomes 10. Then I
